# Code for the topic modelling

Tuto - https://www.youtube.com/watch?v=arMTVXEbV9Q

## Import packages

In [4]:
# nltk
import nltk
# gensim
import gensim
import gensim.corpora as corpora
# spacy
import spacy
#vis
import pyLDAvis
import pyLDAvis.gensim

In [17]:
import pandas as pd

## Import data

In [ ]:
db_init = pd.read_csv("C:/Users/valentin.stuhlfauth/OneDrive - univ-lyon2.fr/2_2025-DUT systematic review/2_Topic modelling/0_code/2025_Pol4PED_systematic_review/2025-02-27_Screening_result.csv",sep=';',encoding='ISO-8859-1')

In [14]:
db_init["Approach"].unique()

array(['Conceptual', 'Methodological', 'Qualitative', nan, 'Reviews',
       'Quantitative', 'Modelling', 'Ingineering'], dtype=object)

In [72]:
db_init=db_init.reset_index()

In [42]:
df_text = pd.DataFrame()
df_text["text_topic"] = data["Article Title"] + " " + data["Abstract"]
df_text = df_text["text_topic"]

## Pretreatment

### Lemmatisation, generation and bigrams

In [44]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)

In [46]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

In [47]:
data_words = gen_words(lemmatization(df_text))

In [93]:
#BIGRAMS AND TRIGRAMS
## threshold (float, optional) – Represent a score threshold for forming the phrases (higher means fewer phrases) (init = 100)
bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=10)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=10)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return ([trigram[bigram[doc]] for doc in texts])

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)

print (data_bigrams_trigrams[0])

['theory', 'paper_analyze', 'relationship', 'low_carbon_city', 'climate_change', 'background', 'global_warming', 'low_carbon_city', 'construction', 'important', 'measure', 'mitigate_climate_change', 'crisis', 'low_carbon_urban', 'planning', 'key', 'technology', 'low_carbon_urban', 'construction', 'research', 'aspect', 'low_carbon_urban', 'planning', 'theory', 'concept', 'practice', 'low', 'zero', 'carbon', 'community', 'low_carbon_city', 'development', 'future', 'sum', 'low_carbon_urban', 'planning', 'research', 'progress', 'domestic', 'overseas', 'put', 'future', 'trend', 'low_carbon_urban', 'planning', 'research', 'aspect', 'demarcation', 'path', 'low_carbon_city', 'low_carbon_urban', 'planning', 'construction', 'popularity', 'low_carbon_technology', 'enforcement', 'mechanism', 'innovation', 'low_carbon_city', 'construction']


### TF-IDF (play with hypervalue)

In [94]:
#TF-IDF REMOVAL
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]
# print (corpus[0][0:20])

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.02
words  = []
words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

In [95]:
list(dict.fromkeys(words))

['use',
 'urban',
 'base',
 'low_carbon',
 'impact',
 'level',
 'city',
 'study',
 'development',
 'policy',
 'model',
 'analysis',
 'paper',
 'promote',
 'carbon_emission',
 'energy',
 'high',
 'reduce',
 'effect',
 'research',
 'challenge',
 'focus',
 'provide',
 'method',
 'also',
 'find',
 'build',
 'potential',
 'role',
 'affect',
 'approach',
 'goal',
 'need',
 'improve',
 'low_carbon_city',
 'datum',
 'different',
 'result',
 'increase',
 'area',
 'system',
 'low',
 'show',
 'such',
 'carbon',
 'develop',
 'more',
 'important',
 'sustainable',
 'large',
 'design',
 'local',
 'contribute',
 'include',
 'however',
 'apply',
 'emission',
 'consider',
 'construction',
 'analyze',
 'economic',
 'green',
 'strategy',
 'explore',
 'support',
 'evidence',
 'achieve',
 'implementation',
 'result_show',
 'take',
 'evaluate',
 'community',
 'concept',
 'innovation',
 'finally',
 'finding',
 'term',
 'transition',
 'comprehensive',
 'reveal',
 'decrease',
 'value',
 'measure',
 'environment

### Manual Thesaurus

In [84]:
# Stop words 
stop_words=[]
stop_words.append(['CO2'])

print(stop_words)

[['CO2']]


In [85]:
# Stop words suppression 
data_bigrams_trigrams_stop = [[mot for mot in doc if not mot in stop_words] for doc in data_bigrams_trigrams]

## Topic modelling global

### Dictionary

In [96]:
from gensim.corpora import Dictionary
dico = Dictionary(data_bigrams_trigrams)
print(dico)

Dictionary<8958 unique tokens: ['aspect', 'background', 'carbon', 'climate_change', 'community']...>


In [97]:
id2word = corpora.Dictionary(data_bigrams_trigrams)

corpus = []
for text in data_bigrams_trigrams:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word[[0][:1][0]]
print (word)

[(0, 2), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 4), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 5), (19, 1)]
aspect


### LDA (init passes=10)

In [98]:
# lissing topic appartenance = alpha (close to 0 = crisp (binary) / more than 10 = lissing of the results) (deffault = 1/num_topics)
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus[:-1],
                                           id2word=id2word,
                                           num_topics=30,
                                           #random_state=100,
                                           update_every=1,
                                           chunksize=10,
                                           passes=5,
                                           alpha="auto")

In [99]:
lda_model.print_topics()[0:2]

[(9,
  '0.000*"outgrow" + 0.000*"superlinear" + 0.000*"definite" + 0.000*"implant" + 0.000*"implantation" + 0.000*"protective" + 0.000*"allometric" + 0.000*"explored" + 0.000*"restrain" + 0.000*"commissioning"'),
 (28,
  '0.000*"outgrow" + 0.000*"superlinear" + 0.000*"definite" + 0.000*"implant" + 0.000*"implantation" + 0.000*"protective" + 0.000*"allometric" + 0.000*"explored" + 0.000*"restrain" + 0.000*"commissioning"')]

In [100]:
lda_model.show_topic(0)

[('energy_consumption', 0.13961035),
 ('plan', 0.11291957),
 ('make', 0.0870801),
 ('climate_change', 0.07089591),
 ('well', 0.07031956),
 ('economy', 0.055855542),
 ('industrial_structure', 0.043653347),
 ('people', 0.04185459),
 ('define', 0.03542925),
 ('opportunity', 0.033407904)]

In [101]:
# serach terms in topic
lda_model.get_term_topics("sharing", minimum_probability=0)

[]

### Vizualizing the Data

In [102]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
14    -0.437563 -0.356539       1        1  28.884722
10     0.026586 -0.543181       2        1  14.169078
12    -0.535057 -0.087773       3        1   9.856105
3      0.349826 -0.379945       4        1   8.899473
2      0.453655 -0.214068       5        1   7.704030
22     0.514118 -0.022484       6        1   6.023334
7      0.476254  0.239191       7        1   5.758902
0      0.180078 -0.396281       8        1   4.576467
29    -0.294192 -0.308478       9        1   3.691438
21    -0.140652 -0.405058      10        1   2.937473
11    -0.431237  0.088637      11        1   2.806688
19    -0.342583  0.317710      12        1   2.179114
1      0.051681 -0.236839      13        1   1.504398
20    -0.087274 -0.078609      14        1   0.512508
17     0.019492  0.273985      15        1   0.330481
15     0.013125  0.140649      16        1   0.011371
16     0.013125  0.140649      17        1   0.011030
5      0.013125  0.140649      18        1   0.011030
18     0.013125  0.140649      19        1   0.011030
8      0.013125  0.140649      20        1   0.011030
13     0.013125  0.140649      21        1   0.011030
6      0.013125  0.140649      22        1   0.011030
4      0.013125  0.140649      23        1   0.011030
23     0.013125  0.140649      24        1   0.011030
24     0.013125  0.140649      25        1   0.011030
25     0.013125  0.140649      26        1   0.011030
26     0.013125  0.140649      27        1   0.011030
27     0.013125  0.140649      28        1   0.011030
28     0.013125  0.140649      29        1   0.011030
9      0.013125  0.140649      30        1   0.011030, topic_info=                Term          Freq         Total Category  logprob  loglift
93             urban  10790.000000  10790.000000  Default  30.0000  30.0000
43   carbon_emission   4595.000000   4595.000000  Default  29.0000  29.0000
44              city  12542.000000  12542.000000  Default  28.0000  28.0000
102           energy   4546.000000   4546.000000  Default  27.0000  27.0000
9        development   5086.000000   5086.000000  Default  26.0000  26.0000
..               ...           ...           ...      ...      ...      ...
22         mechanism      0.002652    590.010791  Topic30  -9.1003  -3.2004
25     paper_analyze      0.002652    104.814377  Topic30  -9.1003  -1.4724
26              path      0.002652    235.912676  Topic30  -9.1003  -2.2837
27          planning      0.002652   1809.248476  Topic30  -9.1003  -4.3209
29          practice      0.002652    274.787843  Topic30  -9.1003  -2.4362

[1451 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
165       1  0.998368     accord
256       2  0.999238    achieve
240      12  0.988204     actual
241      12  0.993907    advance
299       4  0.997427  advantage
...     ...       ...        ...
65        8  0.998561       well
297       5  0.997228      whole
298       6  0.992944       wide
238       2  0.998493       work
239       5  0.997023  worldwide

[255 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[15, 11, 13, 4, 3, 23, 8, 1, 30, 22, 12, 20, 2, 21, 18, 16, 17, 6, 19, 9, 14, 7, 5, 24, 25, 26, 27, 28, 29, 10])

## Topic modelling "Scale and Resources" > 

## Data analysis

In [64]:
# description of topics' space
doc_topics = lda_model.get_document_topics(corpus[:-1])
print(doc_topics)

In [65]:
# "sparce" matrix transformation
from gensim.matutils import corpus2csc
mat_sparce = corpus2csc(doc_topics)

print(mat_sparce)

  (5, 0)	0.026408571749925613
  (6, 0)	0.017804166302084923
  (10, 0)	0.08102951943874359
  (15, 0)	0.028569821268320084
  (17, 0)	0.014475645497441292
  (21, 0)	0.013427573256194592
  (26, 0)	0.16010437905788422
  (29, 0)	0.018302064388990402
  (32, 0)	0.015604554675519466
  (33, 0)	0.05355236306786537
  (34, 0)	0.05800873786211014
  (39, 0)	0.03783108666539192
  (42, 0)	0.08864591270685196
  (46, 0)	0.19876927137374878
  (47, 0)	0.017536601051688194
  (49, 0)	0.10699625313282013
  (5, 1)	0.04468901455402374
  (6, 1)	0.041244301944971085
  (13, 1)	0.021645143628120422
  (15, 1)	0.023561712354421616
  (18, 1)	0.013892151415348053
  (21, 1)	0.013131385669112206
  (24, 1)	0.04393310099840164
  (26, 1)	0.06432025879621506
  (29, 1)	0.03627924993634224
  :	:
  (20, 2025)	0.015533776953816414
  (24, 2025)	0.011222349479794502
  (27, 2025)	0.014651557430624962
  (29, 2025)	0.12063784152269363
  (33, 2025)	0.20882399380207062
  (34, 2025)	0.023475809022784233
  (38, 2025)	0.02848108299076557


In [66]:
# data frame
mat_dt = mat_sparce.T.toarray()
mat_dt

array([[0.        , 0.        , 0.        , ..., 0.0175366 , 0.        ,
        0.10699625],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.08628862],
       [0.        , 0.        , 0.02039475, ..., 0.        , 0.        ,
        0.20102744],
       ...,
       [0.02747214, 0.01926056, 0.        , ..., 0.01365835, 0.        ,
        0.21855922],
       [0.02381448, 0.        , 0.        , ..., 0.05243335, 0.        ,
        0.14668371],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.15873881]])

In [74]:
# data frame with polarities
dfTopic = pd.DataFrame(mat_dt, columns = ["T" + str(i) for i in range(mat_dt.shape[1])])
df_Topic_treatment = dfTopic
df_Topic_treatment['id'] = db_init['index']

print(df_Topic_treatment.head())

    T0        T1        T2   T3   T4        T5        T6   T7        T8   T9  \
0  0.0  0.000000  0.000000  0.0  0.0  0.026409  0.017804  0.0  0.000000  0.0   
1  0.0  0.000000  0.000000  0.0  0.0  0.044689  0.041244  0.0  0.000000  0.0   
2  0.0  0.000000  0.020395  0.0  0.0  0.028749  0.000000  0.0  0.097484  0.0   
3  0.0  0.000000  0.000000  0.0  0.0  0.010393  0.000000  0.0  0.000000  0.0   
4  0.0  0.067151  0.000000  0.0  0.0  0.000000  0.000000  0.0  0.000000  0.0   

   ...  T41       T42  T43  T44       T45       T46       T47  T48       T49  \
0  ...  0.0  0.088646  0.0  0.0  0.000000  0.198769  0.017537  0.0  0.106996   
1  ...  0.0  0.054272  0.0  0.0  0.017556  0.205746  0.000000  0.0  0.086289   
2  ...  0.0  0.016196  0.0  0.0  0.000000  0.262996  0.000000  0.0  0.201027   
3  ...  0.0  0.015630  0.0  0.0  0.000000  0.135810  0.000000  0.0  0.265583   
4  ...  0.0  0.000000  0.0  0.0  0.000000  0.125462  0.000000  0.0  0.047965   

   id  
0   0  
1   1  
2   2  
3   3 